In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#출력 그림 크기 조절
sns.set(rc={'figure.figsize':(12,12)})
plt.style.use('ggplot')
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings
warnings.filterwarnings("ignore")

In [23]:
# 데이터 로드
tr = pd.read_csv('data/train0723(5).3.csv')
te = pd.read_csv('data/test0723.fi.csv')

In [24]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 42 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   code              430 non-null    object 
 1   tot_pop           430 non-null    int64  
 2   region            430 non-null    int64  
 3   num_empty         430 non-null    int64  
 4   sub               430 non-null    int64  
 5   bus               430 non-null    int64  
 6   num_park          430 non-null    int64  
 7   type_new          430 non-null    int64  
 8   prop_type_im1     430 non-null    float64
 9   prop_type_im2     430 non-null    float64
 10  prop_type_im3     430 non-null    float64
 11  prop_type_im4     430 non-null    int64  
 12  prop_type_im5     430 non-null    float64
 13  prop_type_im6     430 non-null    float64
 14  prop_type_im7     430 non-null    float64
 15  prop_type_im8     430 non-null    float64
 16  prop_type_im9     430 non-null    float64
 1

### 4.2 Randomforest


In [25]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [26]:
train_RF = tr.set_index("code")
test_RF = te.set_index("code")
# train_RF["지역"] = train_RF["지역"].astype('category').cat.codes
# test_RF["지역"] = test_RF["지역"].astype('category').cat.codes


In [27]:
train_RF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 430 entries, C1000 to C2692
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tot_pop           430 non-null    int64  
 1   region            430 non-null    int64  
 2   num_empty         430 non-null    int64  
 3   sub               430 non-null    int64  
 4   bus               430 non-null    int64  
 5   num_park          430 non-null    int64  
 6   type_new          430 non-null    int64  
 7   prop_type_im1     430 non-null    float64
 8   prop_type_im2     430 non-null    float64
 9   prop_type_im3     430 non-null    float64
 10  prop_type_im4     430 non-null    int64  
 11  prop_type_im5     430 non-null    float64
 12  prop_type_im6     430 non-null    float64
 13  prop_type_im7     430 non-null    float64
 14  prop_type_im8     430 non-null    float64
 15  prop_type_im9     430 non-null    float64
 16  prop_type_im10    430 non-null    float64
 

In [28]:
x_train = train_RF.drop(columns=["target"])
y_train = train_RF.loc[:,"target"]
x_test = test_RF

In [30]:
model = RandomForestRegressor(n_jobs=-1, random_state=40)

In [31]:
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=40)

In [32]:
pred = model.predict(x_test)

In [33]:
submission = pd.read_csv('data/sample_submission.csv')

In [34]:
submission['num'] = pred

In [35]:
submission

,code,num
0,C1072,536.49
1,C1128,1157.51
2,C1456,474.01
3,C1840,512.00
4,C1332,972.43
...,...,...
145,C2456,248.54
146,C1266,474.95
147,C2152,71.63
148,C1267,361.04


In [36]:
submission.to_csv('sub/baseline0725.csv', index=False)

In [63]:
#특성 선택

In [64]:
corr = train_RF.corr()["등록차량수"].abs().sort_values(ascending=False)
corr.tail(20)

면적_60                  0.195569
면적_50                  0.193437
50대(여자)                0.190405
면적_15                  0.186305
면적_80                  0.185849
공급유형_행복주택              0.179600
자격유형_행복주택_공급대상         0.179600
공가수                    0.178797
20대(남자)                0.177481
지역                     0.124213
버스                     0.116681
자격유형_국민임대/장기전세_공급대상    0.114534
공급유형_공공임대(50년)         0.109707
o60                    0.092509
공급유형_국민임대/장기전세         0.082324
40대(여자)                0.066840
20대(여자)                0.046120
면적_40                  0.043873
u10                    0.011700
지하철                    0.007102
Name: 등록차량수, dtype: float64

In [65]:
corr.head(20)

등록차량수                      1.000000
단지내주차면수                    0.847504
총세대수                       0.594550
총임대가구수                     0.580188
임대료                        0.458075
공급유형_공공임대(5년/10년/분납/분양)    0.405901
임대보증금                      0.346605
p30                        0.329077
공급유형_임대상가                  0.317346
면적_70                      0.312048
자격유형_D                     0.273035
면적_30                      0.269377
면적_100                     0.262505
50대(남자)                    0.253219
자격유형_A                     0.237133
면적_20                      0.234213
40대(남자)                    0.209183
공급유형_영구임대                  0.201856
자격유형_영구임대_공급대상             0.197175
면적_60                      0.195569
Name: 등록차량수, dtype: float64

In [66]:
X = train_RF.copy()
X = X.loc[:,corr.index[0:10]]
X = pd.merge(X,train_RF.loc[:,corr.index[-10:]],how = 'left', on = '단지코드')
X.drop(['등록차량수',  ],axis=1, inplace=True)

#X.drop(['등록차량수',  ],axis=1, inplace=True)
feature_names = X.columns.to_list()

# scaler = StandardScaler()
# X = scaler.fit(X).transform(X)

y = train_RF.loc[:,'등록차량수']

In [67]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 411 entries, C2515 to C2532
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   단지내주차면수                  411 non-null    float64
 1   총세대수                     411 non-null    float64
 2   총임대가구수                   411 non-null    float64
 3   임대료                      411 non-null    float64
 4   공급유형_공공임대(5년/10년/분납/분양)  411 non-null    float64
 5   임대보증금                    411 non-null    float64
 6   p30                      411 non-null    float64
 7   공급유형_임대상가                411 non-null    float64
 8   면적_70                    411 non-null    float64
 9   버스                       411 non-null    float64
 10  자격유형_국민임대/장기전세_공급대상      411 non-null    float64
 11  공급유형_공공임대(50년)           411 non-null    float64
 12  o60                      411 non-null    float64
 13  공급유형_국민임대/장기전세           411 non-null    float64
 14  40대(여자)                  

In [37]:
model = RF = RandomForestRegressor(n_jobs=-1, random_state=300, max_depth =12,min_samples_leaf =2,min_samples_split=2,n_estimators=200)
model.fit(X, y)

X_test = test_RF[feature_names]
pred = RF.predict(X_test)

submission['num'] = pred


NameError: name 'X' is not defined

In [69]:
submission

,code,num
0,C1072,689.939629
1,C1128,1078.657340
2,C1456,547.399774
3,C1840,503.813429
4,C1332,1043.438328
...,...,...
145,C2456,274.734651
146,C1266,575.771749
147,C2152,43.180185
148,C1267,482.600040


In [70]:
#submission.to_csv('sub/baseline0715_RF_cod.csv', index=False)

In [71]:
# train_RF,test_RF를 통해 예측

In [51]:
from catboost import CatBoostRegressor

In [52]:
cat_features = []
target_col = 'target'

In [53]:
from catboost import CatBoostRegressor
catb = CatBoostRegressor(
         cat_features=cat_features,
         loss_function='MAE',
         n_estimators=500, 
         learning_rate=0.05, 
         random_state=42
    )
    
catb.fit(train_RF.drop(columns=[target_col]), train_RF[target_col], verbose=100)

0:	learn: 279.6545337	total: 1.81ms	remaining: 903ms
100:	learn: 90.0021841	total: 128ms	remaining: 504ms
200:	learn: 65.7298787	total: 245ms	remaining: 365ms
300:	learn: 53.4754961	total: 358ms	remaining: 236ms
400:	learn: 44.0071425	total: 473ms	remaining: 117ms
499:	learn: 38.6011279	total: 585ms	remaining: 0us


In [54]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission.head()

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0


In [55]:
sample_submission['num'] = catb.predict(test_RF)

In [56]:
sample_submission

,code,num
0,C1072,730.718192
1,C1128,1189.853718
2,C1456,560.240151
3,C1840,525.315100
4,C1332,1019.199423
...,...,...
145,C2456,222.340313
146,C1266,486.245659
147,C2152,103.261810
148,C1267,434.404767


In [57]:
sample_submission.to_csv('sub/submission_catb_baseline_20210725.csv', index=False)

In [79]:
# 영향도 상위 9개 하위 9개 X, X_test 이용

In [80]:
from catboost import CatBoostRegressor

In [81]:
cat_features = []
target_col = '등록차량수'

In [82]:
X = train_RF.copy()
X = X.loc[:,corr.index[0:10]]
X = pd.merge(X,train_RF.loc[:,corr.index[-10:]],how = 'left', on = '단지코드')

feature_names = X.columns.to_list()
feature_names = feature_names.remove("등록차량수")
# scaler = StandardScaler()
# X = scaler.fit(X).transform(X)

In [83]:
from catboost import CatBoostRegressor
catb = CatBoostRegressor(
         cat_features=cat_features,
         loss_function='MAE',
         n_estimators=500, 
         learning_rate=0.05, 
         random_state=42
    )
    
catb.fit(X.drop(columns=[target_col]), X[target_col], verbose=100)

0:	learn: 275.8287093	total: 1.45ms	remaining: 726ms
100:	learn: 88.9728298	total: 167ms	remaining: 658ms
200:	learn: 68.4239445	total: 319ms	remaining: 474ms
300:	learn: 52.2832180	total: 494ms	remaining: 327ms
400:	learn: 43.3917721	total: 661ms	remaining: 163ms
499:	learn: 38.3946664	total: 814ms	remaining: 0us


In [84]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission.head()

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0


In [85]:
sample_submission['num'] = catb.predict(X_test)

In [86]:
sample_submission

,code,num
0,C1072,707.937210
1,C1128,1137.440992
2,C1456,451.922942
3,C1840,524.290090
4,C1332,1088.544712
...,...,...
145,C2456,250.932651
146,C1266,577.567230
147,C2152,57.514887
148,C1267,445.958561


In [87]:
sample_submission.to_csv('sub/submission_catb_topbot18_20210717.csv', index=False)